In [1]:
import pickle
import json
import os
import numpy as np

artifacts_dir = '/Users/bipulkumar/Documents/mlops_fup_artifacts2'

In [2]:
os.listdir(artifacts_dir)

['modellog.pkl',
 'server.py',
 '.DS_Store',
 'minmaxscaler.pkl',
 'util.py',
 'demo_flask.py',
 'labelencoding.pkl',
 '__pycache__',
 'columns.json']

In [3]:
print(os.path.join(artifacts_dir,'modellog.pkl'))
print(os.path.join(artifacts_dir,'minmaxscaler.pkl'))
print(os.path.join(artifacts_dir,'labelencoding.pkl'))
print(os.path.join(artifacts_dir,'columns.json'))

/Users/bipulkumar/Documents/mlops_fup_artifacts2/modellog.pkl
/Users/bipulkumar/Documents/mlops_fup_artifacts2/minmaxscaler.pkl
/Users/bipulkumar/Documents/mlops_fup_artifacts2/labelencoding.pkl
/Users/bipulkumar/Documents/mlops_fup_artifacts2/columns.json


In [4]:
with open(os.path.join(artifacts_dir,'modellog.pkl') , 'rb') as f:
    _model = pickle.load(f)

In [5]:
_model.coef_

array([[ 0.02382467,  0.35479553,  0.39528663, -0.442547  , -0.04913972,
         0.04022649, -0.81819713,  0.23511056,  3.18026394,  0.22583306]])

In [6]:
with open(os.path.join(artifacts_dir,'minmaxscaler.pkl') , 'rb') as f:
    _scaling = pickle.load(f)

In [7]:
with open(os.path.join(artifacts_dir,'columns.json')) as f:
    _data_columns = json.load(f)['features']

In [8]:
print(_data_columns)

['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']


In [9]:
num_cols = ["ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term"]

cat_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 
            'Property_Area']


In [10]:
# Define the prediction function
def predict(gender, married, dependents, education, selfemployed, applicantincome, loanamount,
            loanamountterm, credithistory, propertyarea):

    # Encode categorical features
    gender_encode = lambda x : 0 if x == 'Female' else 1
    married_encode = lambda x : 0 if x == 'No' else 1
    dependents_encode = lambda x : 0 if x == '0' else 1 if x == '1' else 2 if x == '2' else 3
    education_encode = lambda x : 0 if x == 'Graduate' else 1
    self_employed_encode = lambda x : 0 if x == 'No' else 1
    property_area_encode = lambda x : 0 if x == 'Rural' else 1 if x == 'Semiurban' else 2
    
    gender = gender_encode(gender)
    married = married_encode(married)
    dependents = dependents_encode(dependents)
    education = education_encode(education)
    selfemployed = self_employed_encode(selfemployed)
    credithistory = credithistory
    propertyarea = property_area_encode(propertyarea)
    
    # Log transform numerical features
    applicantincome_log = np.log(applicantincome)
    loanamount_log = np.log(loanamount)
    loanamountterm_log = np.log(loanamountterm)
    
    # Combine all features into a single array
    input_data = np.array([[gender, married, dependents, education, selfemployed, 
                            applicantincome_log, loanamount_log, loanamountterm_log, 
                            credithistory, propertyarea]])

    print(input_data)
    # Apply scaling to the entire input data
    input_data_scaled = _scaling.transform(input_data)
    print(input_data_scaled)

    # Predict using the loaded model
    prediction = _model.predict(input_data_scaled)
    
    print("Predicted Loan Status:", prediction[0])

# Example usage
predict('Male', 'Yes', '0', 'Not Graduate', 'Yes', 5587, 152, 360.0, 1.0, 'Urban')

[[1.         1.         0.         1.         1.         8.62819775
  5.02388052 5.88610403 1.         2.        ]]
[[1.         1.         0.         1.         1.         0.33621421
  0.6492305  0.9220137  1.         1.        ]]
Predicted Loan Status: Y


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [11]:
import pandas as pd

df_test = pd.read_csv('test_loan.csv')

df_test['ApplicantIncome'] = df_test['ApplicantIncome'] + df_test['CoapplicantIncome']
 
df_test = df_test.drop(['Loan_ID','CoapplicantIncome'], axis = 'columns')

df_test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,Yes,0,Graduate,No,5720,110,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,4576,126,360.0,1.0,Urban
2,Male,Yes,2,Graduate,No,6800,208,360.0,1.0,Urban
3,Male,Yes,2,Graduate,No,4886,100,360.0,NaN,Urban
4,Male,No,0,Not Graduate,No,3276,78,360.0,1.0,Urban


In [12]:
df_test.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History',
       'Property_Area'],
      dtype='object')

In [13]:
df_test.values[5]

array(['Male', 'Yes', '0', 'Not Graduate', 'Yes', 5587, 152, 360.0, 1.0,
       'Urban'], dtype=object)

In [14]:
predict('Male', 'Yes', '0', 'Not Graduate', 'Yes', 5587, 152, 360.0, 1.0,'Urban')

Predicted Loan Status: Y


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [12]:
import sklearn

In [13]:
sklearn.__version__

'1.2.2'